In [1]:
import pandas as pd
import numpy as np

V našom projekte budeme využívať 1 dataset, ktorý obsahuje informácie o zápasoch NHL. Postupne vytvoríme 5 rôznych datasetov, na ktorých budeme trénovať našu neurónovú sieť.

V prvom datasete budeme využívať iba jednu tabuľku s názvom game.


In [2]:
gameInput = pd.read_csv("nhl-game-data/game.csv")

In [3]:
gameInput.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11434 entries, 0 to 11433
Data columns (total 16 columns):
game_id                   11434 non-null int64
season                    11434 non-null int64
type                      11434 non-null object
date_time                 11434 non-null object
date_time_GMT             11434 non-null object
away_team_id              11434 non-null int64
home_team_id              11434 non-null int64
away_goals                11434 non-null int64
home_goals                11434 non-null int64
outcome                   11434 non-null object
home_rink_side_start      11140 non-null object
venue                     11434 non-null object
venue_link                11434 non-null object
venue_time_zone_id        11434 non-null object
venue_time_zone_offset    11434 non-null int64
venue_time_zone_tz        11434 non-null object
dtypes: int64(7), object(9)
memory usage: 1.4+ MB


Vidíme, že iba jeden stĺpec obsahuje NaN hodnoty. No tento stĺpec, home_rink_side_start, spolu aj s ďalšími, nebudeme využívať. Preto tieto stĺpce vymažeme.

In [4]:
gameInput1 = gameInput.loc[:,['game_id', 'season', 'date_time_GMT', 'home_team_id', 'away_team_id', 'date_time', 'away_goals', 'home_goals']]

In [5]:
gameInput1

,game_id,season,date_time_GMT,home_team_id,away_team_id,date_time,away_goals,home_goals
0,2011030221,20112012,2012-04-29T19:00:00Z,4,1,2012-04-29,3,4
1,2011030222,20112012,2012-05-01T23:30:00Z,4,1,2012-05-01,4,1
2,2011030223,20112012,2012-05-03T23:30:00Z,1,4,2012-05-03,3,4
3,2011030224,20112012,2012-05-06T23:30:00Z,1,4,2012-05-06,2,4
4,2011030225,20112012,2012-05-08T23:30:00Z,4,1,2012-05-08,3,1
...,...,...,...,...,...,...,...,...
11429,2018030413,20182019,2019-06-02T00:00:00Z,19,6,2019-06-02,7,2
11430,2018030414,20182019,2019-06-04T00:00:00Z,19,6,2019-06-04,2,4
11431,2018030415,20182019,2019-06-07T00:00:00Z,6,19,2019-06-07,2,1
11432,2018030416,20182019,2019-06-10T00:00:00Z,19,6,2019-06-10,5,1


Ďalej si potrebujeme vypočítať priemerný počet gólov vstrelených, aj inkasovaných, počas celej sezóny pre každý tím. Rozdelíme to ešte na domáce a vonkajšie zápasy.

Najskôr si vyberieme len tie stĺpce, ktoré budeme potrebovať  na výpočet priemerného počtu gólov. 
Dáta budeme zhlukovať najskôr podľa tímu, a následne podľa sezóny. Následne nad týmito zhlukmi aplikujeme agregácie, konkrétne vypočítame priemerné počty gólov vstrelených aj inkasovaných, a aj počet domácich aj vonkajších zápasov. 

Výstupom sú dva dataFrame-y, jeden pre domáce zápasy a jeden pre vonkajšie zápasy.

In [6]:
gameAwayValues = gameInput.loc[:,['away_team_id', 'date_time', 'away_goals', 'home_goals', 'season']].groupby(['away_team_id', 'season']) \
       .agg({'date_time':'size', 'away_goals':'mean', 'home_goals':'mean'}) \
       .rename(columns={'date_time':'count'}) \
       .reset_index()
gameHomeValues = gameInput.loc[:,['home_team_id', 'date_time', 'away_goals', 'home_goals', 'season']].groupby(['home_team_id', 'season']) \
       .agg({'date_time':'size', 'away_goals':'mean', 'home_goals':'mean'}) \
       .rename(columns={'date_time':'count'}) \
       .reset_index()
print(gameAwayValues.head(5))
print(gameHomeValues.head(5))

   away_team_id    season  count  away_goals  home_goals
0             1  20102011     41    1.951220    2.634146
1             1  20112012     54    2.574074    2.574074
2             1  20122013     24    1.916667    2.916667
3             1  20132014     41    2.243902    2.829268
4             1  20142015     41    2.195122    3.073171
   home_team_id    season  count  away_goals  home_goals
0             1  20102011     41    2.463415    2.292683
1             1  20112012     52    2.461538    2.846154
2             1  20122013     24    2.458333    2.750000
3             1  20132014     41    2.243902    2.560976
4             1  20142015     41    2.195122    2.219512


Následne potrebujeme vypočítať priemer gólov pre všetky odohrané zápasy počas predchádzajúcej sezóny pre daný tím. 

Najskôr si vytvoríme prázdne dataFrame-y pre domáce a vonkajšie zápasy. Následne vypočítame priemer vstrelených gólov vo všetkých zápasoch a inkasovaných góloch vo všetkých zápasoch v predchádzajúcej sezóne pre každý tím. Keďže táto tabuľka obsahuje dáta od sezóny 2010/2011, tak vo výsledku budú zápasy z tejto sezóny chýbať, pretože nemáme informácie o góloch zo sezóny 2009/2010.

In [7]:
homeTeamStats = pd.DataFrame(columns=['home_team_id', 'season', 'scored', 'conceded'])
awayTeamStats = pd.DataFrame(columns=['away_team_id', 'season', 'scored', 'conceded'])
for index, row in gameAwayValues.iterrows():
    if not gameHomeValues.loc[(gameHomeValues['home_team_id'] == row['away_team_id']) & (gameHomeValues['season'] == row['season'])].empty:
        season = row['season']
        asHomeTeam = gameHomeValues.loc[(gameHomeValues['home_team_id'] == row['away_team_id']) & (gameHomeValues['season'] == row['season'])]
        conceded = (asHomeTeam['away_goals'].values[0]*asHomeTeam['count'].values[0] + row['home_goals']*row['count'])/(asHomeTeam['count'].values[0] + row['count'])
        scored = (asHomeTeam['home_goals'].values[0]*asHomeTeam['count'].values[0] + row['away_goals']*row['count'])/(asHomeTeam['count'].values[0] + row['count'])
        d1 = {'away_team_id': row['away_team_id'], 'season': season, 'conceded': conceded, 'scored': scored}
        d2 = {'home_team_id': asHomeTeam['home_team_id'].values[0], 'season': season, 'conceded': scored, 'scored': conceded}
        homeTeamStats.at[index, :] = d2
        awayTeamStats.at[index, :] = d1
print(homeTeamStats)
print(awayTeamStats)

    home_team_id      season   scored conceded
0              1  2.0102e+07  2.54878  2.12195
1              1  2.0112e+07  2.51887  2.70755
2              1  2.0122e+07   2.6875  2.33333
3              1  2.0132e+07  2.53659  2.40244
4              1  2.0142e+07  2.63415  2.20732
..           ...         ...      ...      ...
267           53  2.0162e+07  3.17073  2.40244
268           53  2.0172e+07  3.12195  2.53659
269           53  2.0182e+07  2.71951  2.59756
270           54  2.0172e+07  2.69608  3.22549
271           54  2.0182e+07   2.8427  3.07865

[272 rows x 4 columns]
    away_team_id      season   scored conceded
0              1  2.0102e+07  2.12195  2.54878
1              1  2.0112e+07  2.70755  2.51887
2              1  2.0122e+07  2.33333   2.6875
3              1  2.0132e+07  2.40244  2.53659
4              1  2.0142e+07  2.20732  2.63415
..           ...         ...      ...      ...
267           53  2.0162e+07  2.40244  3.17073
268           53  2.0172e+07  2.5365

Aj vďaka tomu, že dataset, ktorý sme našli má kvalitné dáta, tak nemusíme, ošetrovať veľa chýb. Hodnoty sú presne také aké sme chceli, tak už s nimi nemusíme nič robiť, a môžeme začať vytvárať náš základný dataset, na ktorom budeme trénovať našu neurónovú sieť.

Vytvoríme si finálny dataFrame, z ktorého budeme následne vytvárať vektory pre našu neurónovú sieť. Tento dataFrame bude obsahovať okrem priemerných počtov gólov v predchádzajúcich sezónach aj počty gólov v posledných 5 spoločných zápasoch.

In [8]:
dataSet1 = pd.DataFrame(columns=['game_id', 'season', 'date_time', 'game1home','game1away', 'game1isHome','game2home', \
                                 'game2away','game2isHome','game3home','game3away','game3isHome','game4home','game4away',\
                                 'game4isHome','game5home','game5away','game5isHome','home_team_id', 'gameHome', 'gameAway', \
                                 'home_scored_avg', 'home_conceded_avg', 'away_team_id', 'away_scored_avg', 'away_conceded_avg', \
                                 'total'])

Nasledujúci kód je plne prispôsobený nášmu riešeniu, a preto sa môže zdať, že je neprehľadný. 

Pre každý zápas najskôr nájdeme priemerné počty gólov v minulých sezónach. Vytvoríme si objekt.  

Následne nájdeme pre tieto dva tímy posledných 5 spoločných zápasov, a z nich vyberieme počet gólov domácich a hostí. Dynamicky ich priradíme do objektu d. Zároveň, domáci tím nemusí byť v predchádzajúcich zápasoch ako domáci tím, tak musíme zaznamenať, či bol ako domáci tím aj v týchto zápasoch. Pri každom zápase je flag gameXisHome. Nakoniec ich uložíme do dataSet1.

In [9]:
j = 0
for index, row in gameInput.iterrows():
    season = row['season'] - 10001
    if not awayTeamStats.loc[(awayTeamStats['away_team_id'] == row['away_team_id']) & (awayTeamStats['season'] == season)].empty:
        if not homeTeamStats.loc[(homeTeamStats['home_team_id'] == row['home_team_id']) & (homeTeamStats['season'] == season)].empty:
            awayTeam = awayTeamStats.loc[(awayTeamStats['away_team_id'] == row['away_team_id']) & (awayTeamStats['season'] == season)]
            homeTeam = homeTeamStats.loc[(homeTeamStats['home_team_id'] == row['home_team_id']) & (homeTeamStats['season'] == season)]
            d = {'game_id': row['game_id'],\
                 'season': row['season'],\
                 'date_time': row['date_time'],\
                 'home_team_id': row['home_team_id'], \
                 'gameHome': row['home_goals'], \
                 'gameAway': row['away_goals'], \
                 'home_scored_avg': homeTeam['scored'].values[0], \
                 'home_conceded_avg': homeTeam['conceded'].values[0], \
                 'away_team_id': row['away_team_id'], \
                 'away_scored_avg': awayTeam['scored'].values[0], \
                 'away_conceded_avg': awayTeam['conceded'].values[0], \
                 'total': row['home_goals'] + row['away_goals']}
            for i in range(1,6):
                d['game' + str(i) + 'home'] = 0
                d['game' + str(i) + 'away'] = 0
                d['game' + str(i) + 'isHome'] = 1
            if not gameInput.loc[((gameInput['date_time_GMT'] < row['date_time_GMT']) & \
                                  ((gameInput['home_team_id'] == row['home_team_id']) & \
                                  (gameInput['away_team_id'] == row['away_team_id'])) | \
                                  ((gameInput['home_team_id'] == row['away_team_id']) & \
                                  (gameInput['away_team_id'] == row['home_team_id'])))].tail(5).empty:
                previousGames = gameInput.loc[((gameInput['date_time_GMT'] < row['date_time_GMT']) & \
                                  ((gameInput['home_team_id'] == row['home_team_id']) & \
                                  (gameInput['away_team_id'] == row['away_team_id'])) | \
                                  ((gameInput['home_team_id'] == row['away_team_id']) & \
                                  (gameInput['away_team_id'] == row['home_team_id'])))].tail(5)
                i = 1
                for index1, row1 in previousGames.iterrows():
                    if (row1.home_team_id == row.home_team_id):
                        d['game' + str(i) + 'home'] = row1['home_goals']
                        d['game' + str(i) + 'away'] = row1['away_goals']
                        d['game' + str(i) + 'isHome'] = 1
                    else:
                        d['game' + str(i) + 'home'] = row1['away_goals']
                        d['game' + str(i) + 'away'] = row1['home_goals']
                        d['game' + str(i) + 'isHome'] = 0
                    i += 1
                    
            dataSet1.at[index, :] = d
    

Získame výsledný dataframe, ktorého časť dát môžeme vidieť v nasledujúcom bloku. Ešte predtým ich zoradíme podľa dátumu.

In [10]:
dataSet1 = dataSet1.sort_values(by=['date_time'])
dataSet1

,game_id,season,date_time,game1home,game1away,game1isHome,game2home,game2away,game2isHome,game3home,...,game5isHome,home_team_id,gameHome,gameAway,home_scored_avg,home_conceded_avg,away_team_id,away_scored_avg,away_conceded_avg,total
11049,2011020002,20112012,2011-10-06,0,2,0,1,4,0,5,...,0,10,2,0,3.06098,2.65854,8,2.61798,2.53933,2
9192,2011020001,20112012,2011-10-06,1,2,1,3,0,0,2,...,0,6,1,2,2.31776,3.05607,4,3.09677,2.80645,3
10104,2011020007,20112012,2011-10-07,0,1,0,3,1,0,4,...,1,7,4,1,2.82022,2.95506,24,2.94318,2.92045,5
11113,2011020004,20112012,2011-10-07,4,3,0,1,5,0,2,...,1,12,1,5,2.91463,2.87805,14,3.06,2.85,6
10684,2011020009,20112012,2011-10-07,2,3,0,2,4,0,5,...,0,26,3,2,2.47727,2.71591,3,2.77011,2.42529,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11429,2018030413,20182019,2019-06-02,2,4,1,2,4,0,3,...,0,19,2,7,2.70732,2.7561,6,3.30851,2.67021,9
11430,2018030414,20182019,2019-06-04,2,4,0,3,2,0,2,...,0,19,4,2,2.70732,2.7561,6,3.30851,2.67021,6
11431,2018030415,20182019,2019-06-07,4,2,1,2,3,1,7,...,0,6,1,2,2.67021,3.30851,19,2.7561,2.70732,3
11432,2018030416,20182019,2019-06-10,3,2,0,2,7,1,4,...,0,19,1,5,2.70732,2.7561,6,3.30851,2.67021,6


## Dátová analýza

Priemerný počet vstrelených a inkasovaných gólov na zápas

In [11]:
print(homeTeamStats['scored'].mean().round(4))
print(homeTeamStats['conceded'].mean().round(4))

2.8069
2.7798


Priemerný počet gólov na zápas

In [12]:
(gameInput1.away_goals + gameInput1.home_goals).mean()

5.5885079587196085

Vidíme, že priemerný počet gólov na zápas je 5.59, čo po zaokrúhlení je 6 gólov na zápas. 

In [13]:
gameInput1.loc[(gameInput1.home_goals + gameInput1.away_goals) == 6].game_id.count()

1191

Teda naša úspešnosť, keby sme tipovali počet vstrelených gólov v zápase na základe priemerného počtu gólov, by bola  (1191 / 11 434)*100 = 10.42 %  

## Tvorba Vektoru

Najskôr si vymažeme nepotrebné stĺpce

In [16]:
dataSetFinal = dataSet1.drop('game_id',1).drop('date_time',1)

In [17]:
dataSetFinal

,season,game1home,game1away,game1isHome,game2home,game2away,game2isHome,game3home,game3away,game3isHome,...,game5isHome,home_team_id,gameHome,gameAway,home_scored_avg,home_conceded_avg,away_team_id,away_scored_avg,away_conceded_avg,total
11049,20112012,0,2,0,1,4,0,5,4,0,...,0,10,2,0,3.06098,2.65854,8,2.61798,2.53933,2
9192,20112012,1,2,1,3,0,0,2,1,0,...,0,6,1,2,2.31776,3.05607,4,3.09677,2.80645,3
10104,20112012,0,1,0,3,1,0,4,2,0,...,1,7,4,1,2.82022,2.95506,24,2.94318,2.92045,5
11113,20112012,4,3,0,1,5,0,2,4,1,...,1,12,1,5,2.91463,2.87805,14,3.06,2.85,6
10684,20112012,2,3,0,2,4,0,5,4,0,...,0,26,3,2,2.47727,2.71591,3,2.77011,2.42529,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11429,20182019,2,4,1,2,4,0,3,2,0,...,0,19,2,7,2.70732,2.7561,6,3.30851,2.67021,9
11430,20182019,2,4,0,3,2,0,2,7,1,...,0,19,4,2,2.70732,2.7561,6,3.30851,2.67021,6
11431,20182019,4,2,1,2,3,1,7,2,0,...,0,6,1,2,2.67021,3.30851,19,2.7561,2.70732,3
11432,20182019,3,2,0,2,7,1,4,2,1,...,0,19,1,5,2.70732,2.7561,6,3.30851,2.67021,6


In [27]:
Rozdelíme si dataset na 2 množiny, testovaciu a trénovaciu.

In [18]:
dataTrain1 = dataSetFinal.loc[dataSetFinal.season != 20182019]
dataTest1 = dataSetFinal.loc[dataSetFinal.season == 20182019]

Následne oddelíme z testovacej aj trénovacej množiny očakávané výsledky. Obidve množiny ešte normalizujeme na hodnoty od 0 po 1, na základe maximálnych hodnôt v jednotlivých stĺpcoch.

In [20]:
dataSet1TrainX = dataTrain1.drop('season',1).drop('total',1).drop('gameHome',1).drop('gameAway',1)
dataSet1TrainX = dataSet1TrainX/dataSet1TrainX.max().astype(np.float64)
dataSet1TrainY = dataTrain1.total
dataSet1TestX = dataTest1.drop('season',1).drop('total',1).drop('gameHome',1).drop('gameAway',1)
dataSet1TestX = dataSet1TestX/dataSet1TestX.max().astype(np.float64)
dataSet1TestY = dataTest1.total

Tieto množiny si uložíme do .csv súborov.

In [21]:
dataSet1TrainX.to_csv('.\datasets\dataSet1TrainX.csv', encoding='utf-8', index=False)
dataSet1TrainY.to_csv('.\datasets\dataSet1TrainY.csv', encoding='utf-8',index=False)
dataSet1TestX.to_csv('.\datasets\dataSet1TestX.csv', encoding='utf-8', index=False)
dataSet1TestY.to_csv('.\datasets\dataSet1TestY.csv', encoding='utf-8', index=False)


C:\Users\Filip\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
C:\Users\Filip\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  after removing the cwd from sys.path.
